In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
from xgcm import Grid
import numpy as np
import pandas as pd
from scipy import signal

import IPython.display
import cmocean as cm
import sys, os, glob
import warnings
warnings.simplefilter("ignore") 

from IPython.display import Image

from dask.distributed import Client

from matplotlib.patches import Rectangle

In [ ]:
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['lines.linewidth'] = 2.0
matplotlib.rcParams['font.size'] = 15
letlab = ['(a)','(b)','(c)','(d)','(e)','(f)','(g)','(h)',
          '(i)','(j)','(k)','(l)','(m)','(n)','(o)','(p)',
          '(q)','(r)','(s)','(t)','(u)','(v)','(w)','(x)','(y)','(z)']

In [ ]:
# Load workers:
client = Client(n_workers=8)
client

## Load datasets

In [ ]:
base = '/g/data/e14/rmh561/croco/'
base_data = base + 'archive/'
base_analysis = base + '/analysis/'

os.chdir(base_analysis + 'PAC12_75_cpl')

In [ ]:
# define runs:
expdict = {
           'CROCO-WRF Control-E1':{'exp':'PAC12_75_cpl','expn':'exp02','CPL':True}
#          ,'CROCO-WRF Spinup':{'exp':'PAC12_75_cpl','expn':'exp01','CPL':True}
#           ,'CROCO-WRF Smooth-SST-nomask':{'exp':'PAC12_75_cpl','expn':'exp03','CPL':True}
#          ,'CROCO-WRF Smooth-SST-E2':{'exp':'PAC12_75_cpl','expn':'exp04','CPL':True}
           ,'CROCO-WRF NoCFB':{'exp':'PAC12_75_cpl','expn':'exp06','CPL':True}
#           ,'CROCO-WRF NoMesoCFB-nomask':{'exp':'PAC12_75_cpl','expn':'exp07','CPL':True}
           ,'CROCO-WRF Control-E2':{'exp':'PAC12_75_cpl','expn':'exp08','CPL':True}
           ,'CROCO-WRF Control-E3':{'exp':'PAC12_75_cpl','expn':'exp09','CPL':True}
           ,'CROCO-WRF Control-E4':{'exp':'PAC12_75_cpl','expn':'exp10','CPL':True}
           ,'CROCO-WRF Control-E5':{'exp':'PAC12_75_cpl','expn':'exp11','CPL':True}
           ,'CROCO-WRF Smooth-SST':{'exp':'PAC12_75_cpl','expn':'exp12','CPL':True}
           ,'CROCO-WRF NoMesoCFB':{'exp':'PAC12_75_cpl','expn':'exp13','CPL':True}
           ,'CROCO Control':{'exp':'PAC12_75','expn':'exp11','CPL':False}
#          ,'CROCO PP81':{'exp':'PAC12_75','expn':'exp12','CPL': False}
          }
cont = 'CROCO-WRF Control-E1'

In [ ]:
# Load CROCO data:
data_typs = ['mon','day']#'3day','day','ts_3day']
dropvars = {'mon':['omega','u_sq','v_sq','salt_sq','AKv'],
            'day':[],
            '3day':[],
            'ts_3day':[]}
dropyears = ['2014']
#dropyears = ['2015','2016','2017','2018']

data = {}
for nam in expdict.keys():
    print(nam)
    exp = expdict[nam]['exp']
    expn = expdict[nam]['expn']
    data[nam] = {}
    for typ in data_typs:
        try:
            files = sorted(glob.glob(base_data + exp + '/' + exp + '_' + expn + '/*/croco_out_' + typ + '.nc'))
            for yr in dropyears:
                files = [file for file in files if yr not in file]
            data[nam][typ] = xr.open_mfdataset(files, concat_dim="time_counter", combine="nested",data_vars='minimal', coords='minimal', compat='override',drop_variables=dropvars[typ],parallel=True,chunks={'time_counter':1}).rename({'time_counter':'time'})
        except:
            data[nam][typ] = None
gfiles = glob.glob(base_data + exp + '/' + exp + '_' + expn + '/*/croco_out_grd.nc')
grd = xr.open_dataset(gfiles[0],chunks={'y_rho':100,'x_rho':100})

In [ ]:
# Deal with CROCO coordinates:
for nam in expdict.keys():
    print(nam)
    for d in data[nam].values():
        if d is not None:
            try:
                d["x_rho"] = d.nav_lon_rho.isel(y_rho=0)
                d["y_rho"] = d.nav_lat_rho.isel(x_rho=0)
                d["x_w"] = d.nav_lon_rho.isel(y_rho=0).rename({'x_rho':'x_w'})
                d["y_w"] = d.nav_lat_rho.isel(x_rho=0).rename({'y_rho':'y_w'})
                d["x_u"] = d.nav_lon_u.isel(y_u=0)
                d["y_u"] = d.nav_lat_u.isel(x_u=0)
                d["x_v"] = d.nav_lon_v.isel(y_v=0)
                d["y_v"] = d.nav_lat_v.isel(x_v=0)
                d = d.set_coords({'x_rho','y_rho','x_u','y_u','x_v','y_v','x_w','y_w'})
                
            except:
                d["x_rho"] = d.nav_lon.isel(y=0)
                d["y_rho"] = d.nav_lat.isel(x=0)
                d = d.set_coords({'x_rho','y_rho'})
                
            try:
                d["z_rho"] = d.z_rho.mean('time').fillna(0.)
                d = d.set_coords({'z_rho'})
                
            except:
                pass

# Generate xgcm grid object:
nam = 'CROCO-WRF Control-E1'
grid = Grid(data[nam]['mon'],coords={"x":{"center":"x_rho","inner":"x_u"},
                         "y":{"center":"y_rho","inner":"y_v"},
                         "s":{"center":"s_rho","outer":"s_w"}},periodic=False)

In [ ]:
# Load WRF data:
data_typs = {'day':'3d_1D'}#,'hr':'3d_1H','day4d':'out'}

for nam in expdict.keys():
    if expdict[nam]['CPL']:
        print(nam)
        exp = expdict[nam]['exp']
        expn = expdict[nam]['expn']
        for typ in data_typs.keys():
            try:
                files = sorted(glob.glob(base_data + exp + '/' + exp + '_' + expn + '/*/wrf' + data_typs[typ] + '*.nc'))
                for yr in dropyears:
                    files = [file for file in files if yr not in file]
                data[nam]['wrf_' + typ] = xr.open_mfdataset(files, concat_dim="time_counter", combine="nested",data_vars='minimal', coords='minimal', compat='override',parallel=True,chunks={'time_counter':1}).rename({'time_counter':'time'})
            except:
                data[nam]['wrf_' + typ] = None
        if data[nam]['wrf_day'] is not None:
            lon = data[nam]['wrf_day'].nav_lon.isel(y=0)
            lon = np.where(np.less(lon,0.).values,lon+360.,lon)
            lat = data[nam]['wrf_day'].nav_lat.isel(x=0).values
            for typ in data_typs:
                if data[nam]['wrf_' + typ] is not None:
                    data[nam]['wrf_' + typ]["x"] = lon
                    data[nam]['wrf_' + typ]["y"] = lat

In [ ]:
# Set limit parameters for observations:
for nam in expdict.keys():
    try:
        minyr  = data[nam]['mon'].time.min('time').values.astype('datetime64[Y]').astype(int)+1970
        maxyr  = data[nam]['mon'].time.max('time').values.astype('datetime64[Y]').astype(int)+1970
        print(nam + ' minyr=' + str(minyr) + ' maxyr=' + str(maxyr))
    except:
        print('No data for ' + nam)
nam = 'CROCO-WRF Control-E1'
minyr  = data[nam]['mon'].time.min('time').values.astype('datetime64[Y]').astype(int)+1970
maxyr  = data[nam]['mon'].time.max('time').values.astype('datetime64[Y]').astype(int)+1970
minlon = data[nam]['mon'].x_rho.min('x_rho').values
maxlon = data[nam]['mon'].x_rho.max('x_rho').values
minlat = data[nam]['mon'].y_rho.min('y_rho').values
maxlat = data[nam]['mon'].y_rho.max('y_rho').values

In [ ]:
# # Limit simulations to same years:
# data_typs = ['mon','3day','day','ts_3day']
# for nam in expdict.keys():
#     for typ in data_typs:
#         try:
#             data[nam][typ] = data[nam][typ].sel(time=slice(str(minyr) + '-01-01',str(maxyr) + '-12-31'))
#         except:
#             pass
        
# data_typs = {'day':'3d_1D','hr':'3d_1H','mon':'out'}
# for nam in expdict.keys():
#     if expdict[nam]['CPL']:
#         for typ in data_typs.keys():
#             try:
#                 data[nam]['wrf_' + typ] = data[nam]['wrf_' + typ].sel(time=slice(str(minyr) + '-01-01',str(maxyr) + '-12-31'))
#             except:
#                 pass     

In [ ]:
# Observational SST:
files = []
for yr in np.arange(minyr,maxyr+1,1):
    files += sorted(glob.glob('/g/data/ua8/NOAA_OISST/AVHRR/v2-1_modified/*_' + str(yr) + '.nc'))
OISST = xr.open_mfdataset(files,concat_dim="time", combine="nested",
                  data_vars='minimal', coords='minimal', compat='override',parallel=True).isel(zlev=0)

OISST = OISST.sel(lon=slice(minlon,maxlon)).sel(lat=slice(minlat,maxlat))
OISST = OISST.rename({'lon':'x_rho','lat':'y_rho'})

In [ ]:
# Observational SSH:
files = []
for yr in np.arange(minyr,maxyr+1,1):
    files += sorted(glob.glob('/g/data/ua8/CMEMS_SeaLevel/v2019/' + str(yr) + '/*.nc'))
CMEMS = xr.open_mfdataset(files,concat_dim="time", combine="nested",
                  data_vars='minimal', coords='minimal', compat='override',parallel=True)

CMEMS = CMEMS.sel(longitude=slice(minlon,maxlon)).sel(latitude=slice(minlat,maxlat))
CMEMS = CMEMS.rename({'longitude':'x_rho','latitude':'y_rho'})

In [ ]:
# SODA reanalysis:
files = []
for yr in np.arange(minyr,maxyr+1,1):
    files += sorted(glob.glob('/g/data/e14/rmh561/croco/data/DATA/SODA342_PAC12_75/soda3.4.2_5dy_ocean_reg_' + str(yr) + '_*.nc'))
SODA = xr.open_mfdataset(files,concat_dim="time", combine="nested",
                  data_vars='minimal', coords='minimal', compat='override',parallel=True)

SODA = SODA.sel(xt_ocean=slice(minlon,maxlon)).sel(yt_ocean=slice(minlat,maxlat)).sel(xu_ocean=slice(minlon,maxlon)).sel(yu_ocean=slice(minlat,maxlat))
SODA = SODA.rename({'xt_ocean':'x_rho','yt_ocean':'y_rho'})

In [ ]:
# ACCESS-OM2 data:
import cosima_cookbook as cc
session = cc.database.create_session()
from collections import OrderedDict
exptdict = OrderedDict([
    ('1degIAF',   {'model':'ACCESS-OM2 IAF', 'expt':'1deg_jra55_iaf_omip2_cycle6', 'itime':pd.datetime(minyr,1,1), 'ftime':pd.datetime(maxyr,12,31)}),
    ('025degIAF', {'model':'ACCESS-OM2-025 IAF', 'expt':'025deg_jra55_iaf_omip2_cycle6', 'itime':pd.datetime(minyr,1,1), 'ftime':pd.datetime(maxyr,12,31)}),
    ('01degIAF', {'model':'ACCESS-OM2-01 IAF', 'expt':'01deg_jra55v140_iaf_cycle3', 'itime':pd.datetime(minyr,1,1), 'ftime':pd.datetime(maxyr,12,31)}),
])

# Domain Schematic:

In [ ]:
time = '2016-11-01'
plt.figure(figsize=(10,5))

# EPAC only:
xlims = [115.,300.]
ylims = [-35.,35.]
vlims = [18.,30.]

# EPAC and BPAC:
xlims = [100.,300.]
ylims = [-35.,35.]
vlims = [18.,30.]

ps = OISST.sst.sel(x_rho=slice(xlims[0],xlims[1])).sel(y_rho=slice(ylims[0],ylims[1])).sel(time=time).plot(cmap=cm.cm.thermal,vmin=vlims[0],vmax=vlims[1],cbar_kwargs={'label':''})

data['CROCO-WRF Control']['day'].temp_surf.sel(time=time).plot(cmap=cm.cm.thermal,vmin=vlims[0],vmax=vlims[1],add_colorbar=False)
#data['wrf_day'].WSPD10.sel(time=time).isel(time=0).plot.contour(levels=np.arange(0.,25.,2.),colors='k')
plt.gca().set_facecolor('k')
plt.gca().set_xlim(xlims)
plt.gca().set_ylim(ylims)
plt.gca().add_patch(Rectangle((minlon, minlat), maxlon-minlon, maxlat-minlat,
             edgecolor = 'green',
             fill=False,
             lw=1))
plt.gca().text(minlon+1.,minlat+1.,'EPAC',color='green',fontsize=20)

# BPAC:
BPACminlon = 110.
BPACmaxlon = 290.
BPACminlat = -30.
BPACmaxlat = 30.
plt.gca().add_patch(Rectangle((BPACminlon, BPACminlat), BPACmaxlon-BPACminlon, BPACmaxlat-BPACminlat,
             edgecolor = 'green',
             fill=False,
             lw=1))
plt.gca().text(BPACminlon+1.,BPACminlat+1.,'BPAC',color='green',fontsize=20)

N34minlon = 190.
N34maxlon = 240.
N34minlat = -5.
N34maxlat = 5.
plt.gca().add_patch(Rectangle((N34minlon, N34minlat), N34maxlon-N34minlon, N34maxlat-N34minlat,
             edgecolor = 'black',
             linestyle = 'dashed',
             fill=False,
             lw=1))
plt.gca().text(BPACminlon+1.,BPACminlat+1.,'BPAC',color='green',fontsize=20)

plt.title('OISST and CROCO-WRF SST ' + time + ' ($^\circ$C)',fontsize=15)
plt.xlabel('Longitude ($^\circ$E)',fontsize=15)
plt.ylabel('Latitude ($^\circ$N)',fontsize=15)
for item in (plt.gca().get_xticklabels() + plt.gca().get_yticklabels()):
    item.set_fontsize(15)
plt.tight_layout()

plt.savefig('Domain_Fig.png',dpi=250)

# Snapshots, short periods of SST and surface heat fluxes - CROCO-WRF Control and Smooth-SST

## Nice snapshot smoothing figure:

In [ ]:
#Calculations:
time = '2016-11-01'
tlims = [20.,29.]

fig,axes = plt.subplots(nrows=2,ncols=2,figsize=(20,10),facecolor='w')

# SST:
#(data['CROCO-WRF Control-E1']['wrf_day'].SST-273.15).sel(time=time).plot(ax=axes[0][0],cmap='RdBu_r',vmin=tlims[0],vmax=tlims[1])
data['CROCO-WRF Smooth-SST']['day'].temp_surf.sel(time=time).plot(ax=axes[0][0],cmap='RdBu_r',vmin=tlims[0],vmax=tlims[1],cbar_kwargs={'label':'$^\circ$C'},extend='both')
wrfSST = data['CROCO-WRF Smooth-SST']['wrf_day'].SST.sel(time=time)-273.15
wrfSST = wrfSST.where(wrfSST > 10.)
wrfSST.plot(ax=axes[0][1],cmap='RdBu_r',vmin=tlims[0],vmax=tlims[1],cbar_kwargs={'label':'$^\circ$C'},extend='both')

# V:
data['CROCO-WRF NoMesoCFB']['day'].v_surf.sel(time=time).plot(ax=axes[1][0],cmap='RdBu_r',vmin=-1.,vmax=1.,cbar_kwargs={'label':'ms$^{-1}$'},extend='both')
wrfV = data['CROCO-WRF NoMesoCFB']['wrf_day'].VOCE.sel(time=time)
wrfV = wrfV.where(wrfV != 0.)
wrfV.plot(ax=axes[1][1],cmap='RdBu_r',vmin=-1.,vmax=1.,cbar_kwargs={'label':'ms$^{-1}$'},extend='both')

# Add Gaussian smoother and mask:
x = data['CROCO-WRF Smooth-SST']['wrf_day'].SST.x
gausy = 5.*np.exp(-.5*(x-240.)**2./(24.*1./4.)**2.)
c = xr.open_dataset('/g/data/e14/rmh561/croco/data/OASIS_FILES/PAC12_75_cpl/smooth_mask_tanh.nc')
c = c.assign_coords({"y":data['CROCO-WRF Smooth-SST']['wrf_day'].SST.x.values,"x":data['CROCO-WRF Smooth-SST']['wrf_day'].SST.y.values})

axes[0][1].plot(x,gausy,'--k')
axes[1][1].plot(x,gausy,'--k')
c.SMOOTHMASK.plot.contour(x='y',y='x',ax=axes[0][1],levels=np.arange(0.25,1.25,0.25),colors='k',linestyles='dashed')
c.SMOOTHMASK.plot.contour(x='y',y='x',ax=axes[1][1],levels=np.arange(0.25,1.25,0.25),colors='k',linestyles='dashed')

for axs in axes.flatten():
    axs.set_facecolor('k')
    axs.set_xlabel('Longitude ($^\circ$E)')
    axs.set_ylabel('Latitude ($^\circ$N)')

axes[0][0].set_title('(a) NoTFB CROCO SST')
axes[0][1].set_title('(b) NoTFB WRF SST')
axes[1][0].set_title('(c) NoMesoCFB CROCO Meridional Velocity')
axes[1][1].set_title('(d) NoMesoCFB WRF Meridional Velocity')
plt.tight_layout()

plt.savefig('SST_Example_2016-11.png',dpi=100)

## Other snapshot figures

In [ ]:
#Calculations:
time = '2016-11-01'
#time = '2014-11-01'

fig,axes = plt.subplots(nrows=2,ncols=2,figsize=(20,10),facecolor='w')
data['CROCO-WRF Control-E2']['wrf_day'].VOCE.sel(time=time).plot(ax=axes[0][0],cmap='RdBu_r',vmin=-1.,vmax=1.)
axes[0][0].set_title('CROCO-WRF Control WRF v ' + time)
data['CROCO-WRF NoMesoCFB']['wrf_day'].VOCE.sel(time=time).plot(ax=axes[0][1],cmap='RdBu_r',vmin=-1.,vmax=1.)
axes[0][1].set_title('CROCO-WRF Smooth-SST WRF V ' + time)
data['CROCO-WRF Control-E2']['day'].v_surf.sel(time=time).plot(ax=axes[1][0],cmap='RdBu_r',vmin=-1.,vmax=1.)
axes[1][0].set_title('CROCO-WRF Control CROCO V ' + time)
data['CROCO-WRF NoMesoCFB']['day'].v_surf.sel(time=time).plot(ax=axes[1][1],cmap='RdBu_r',vmin=-1.,vmax=1.)
axes[1][1].set_title('CROCO-WRF Smooth-SST CROCO V' + time)

plt.tight_layout()

#plt.savefig('Smooth-SST/V_Example_Masked_2014-11.png',dpi=100)
#plt.savefig('CFB_Figures/V_Example_Masked_2014-11.png',dpi=100)

In [ ]:
# Flux Calculations:
#times = ['2016-11-01','2016-11-01']
times = ['2016-11-01','2016-11-12']

fig,axes = plt.subplots(nrows=5,ncols=3,figsize=(30,25),facecolor='w')

# CROCO-only SST snapshots for sensible heat flux:
SST = data['CROCO Control']['day'].temp_surf.sel(time=slice(times[0],times[1])).mean('time')
OBS_SST = OISST.sst.sel(time=slice(times[0],times[1])).mean('time')

# Total heat flux:
data['CROCO-WRF Control']['day'].shflx.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[0][0],cmap='RdBu_r',vmin=-250.,vmax=250.)
axes[0][0].set_title('CROCO-WRF Control CROCO $Q_{net}$ ' + times[0] + ' ' + times[1])
data['CROCO-WRF Smooth-SST']['day'].shflx.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[0][1],cmap='RdBu_r',vmin=-250.,vmax=250.)
axes[0][1].set_title('CROCO-WRF Smooth-SST CROCO $Q_{net}$ ' + times[0] + ' ' + times[1])
data['CROCO Control']['day'].shflx.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[0][2],cmap='RdBu_r',vmin=-250.,vmax=250.)
axes[0][2].set_title('CROCO Control CROCO $Q_{net}$ ' + times[0] + ' ' + times[1])

# Sensible heat flux:
SHFLX = -data['CROCO-WRF Control']['wrf_day'].HFX
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[1][0],cmap='RdBu_r',vmin=-50.,vmax=50.)
axes[1][0].set_title('CROCO-WRF Control WRF Sensible HF ' + times[0] + ' ' + times[1])
SHFLX = -data['CROCO-WRF Smooth-SST']['wrf_day'].HFX
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[1][1],cmap='RdBu_r',vmin=-50.,vmax=50.)
axes[1][1].set_title('CROCO-WRF Smooth-SST WRF Sensible HF ' + times[0] + ' ' + times[1])
data['CROCO Control']['day'].shflx_sen.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[1][2],cmap='RdBu_r',vmin=-50.,vmax=50.)
axes[1][2].set_title('CROCO Control CROCO Sensible HF ' + times[0] + ' ' + times[1])

SST.plot.contour(ax=axes[1][2],levels=[25.],colors='k')
OBS_SST.plot.contour(ax=axes[1][2],levels=[25.],colors='k',linestyles='dashed')

# Latent heat flux:
SHFLX = -data['CROCO-WRF Control']['wrf_day'].LH
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[2][0],cmap='RdBu_r',vmin=-200.,vmax=200.)
axes[2][0].set_title('CROCO-WRF Control WRF Latent HF ' + times[0] + ' ' + times[1])
SHFLX = -data['CROCO-WRF Smooth-SST']['wrf_day'].LH
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[2][1],cmap='RdBu_r',vmin=-200.,vmax=200.)
axes[2][1].set_title('CROCO-WRF Smooth-SST WRF Latent HF ' + times[0] + ' ' + times[1])
data['CROCO Control']['day'].shflx_lat.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[2][2],cmap='RdBu_r',vmin=-200.,vmax=200.)
axes[2][2].set_title('CROCO Control CROCO Latent HF ' + times[0] + ' ' + times[1])

# Shortwave heat flux:
data['CROCO-WRF Control']['day'].swrad.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[3][0],cmap='RdBu_r',vmin=-350.,vmax=350.)
axes[3][0].set_title('CROCO-WRF Control CROCO SW Radiation ' + times[0] + ' ' + times[1])
data['CROCO-WRF Smooth-SST']['day'].swrad.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[3][1],cmap='RdBu_r',vmin=-350.,vmax=350.)
axes[3][1].set_title('CROCO-WRF Smooth-SST CROCO SW Radiation ' + times[0] + ' ' + times[1])
data['CROCO Control']['day'].radsw.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[3][2],cmap='RdBu_r',vmin=-350.,vmax=350.)
axes[3][2].set_title('CROCO Control CROCO SW Radiation ' + times[0] + ' ' + times[1])

# longwave heat flux:
SHFLX = data['CROCO-WRF Control']['wrf_day'].GLW-5.67e-8*(data['CROCO-WRF Control']['wrf_day'].SST)**4
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[4][0],cmap='RdBu_r',vmin=-100.,vmax=100.)
axes[4][0].set_title('CROCO-WRF Control WRF Longwave HF ' + times[0] + ' ' + times[1])
SHFLX = data['CROCO-WRF Smooth-SST']['wrf_day'].GLW-5.67e-8*(data['CROCO-WRF Smooth-SST']['wrf_day'].SST)**4
SHFLX.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[4][1],cmap='RdBu_r',vmin=-100.,vmax=100.)
axes[4][1].set_title('CROCO-WRF Smooth-SST WRF Longwave HF ' + times[0] + ' ' + times[1])
data['CROCO Control']['day'].shflx_rlw.sel(time=slice(times[0],times[1])).mean('time').plot(ax=axes[4][2],cmap='RdBu_r',vmin=-100.,vmax=100.)
axes[4][2].set_title('CROCO Control CROCO Longwave HF ' + times[0] + ' ' + times[1])

plt.tight_layout()

#plt.savefig('Smooth-SST/Surface_Fluxes_Example_2016-11-01.png',dpi=100)
#plt.savefig('Smooth-SST/Surface_Fluxes_Example_2016-11-01_to_2016-11-12.png',dpi=100)

In [ ]:
# Flux Calculations:
times = ['2016-11-01','2016-11-01']
times = ['2016-11-01','2016-11-30']

fig,axes = plt.subplots(nrows=5,ncols=1,figsize=(10,25),facecolor='w')

axs = axes.reshape(-1)

# Total heat flux:
(data['CROCO-WRF Smooth-SST']['day'].shflx.sel(time=slice(times[0],times[1])).mean('time')-data['CROCO-WRF Control']['day'].shflx.sel(time=slice(times[0],times[1])).mean('time')).plot(ax=axes[0],cmap='RdBu_r',vmin=-50.,vmax=50.)
axes[0].set_title('CROCO-WRF Control CROCO $Q_{net}$ ' + times[0] + ' ' + times[1])

# Sensible heat flux:
SHFLX1 = -data['CROCO-WRF Control']['wrf_day'].HFX.sel(time=slice(times[0],times[1])).mean('time')
SHFLX2 = -data['CROCO-WRF Smooth-SST']['wrf_day'].HFX.sel(time=slice(times[0],times[1])).mean('time')
(SHFLX2-SHFLX1).plot(ax=axes[1],cmap='RdBu_r',vmin=-20.,vmax=20.)
axes[1].set_title('CROCO-WRF Control WRF Sensible HF ' + times[0] + ' ' + times[1])

# Latent heat flux:
SHFLX1 = -data['CROCO-WRF Control']['wrf_day'].LH.sel(time=slice(times[0],times[1])).mean('time')
SHFLX2 = -data['CROCO-WRF Smooth-SST']['wrf_day'].LH.sel(time=slice(times[0],times[1])).mean('time')
(SHFLX2-SHFLX1).plot(ax=axes[2],cmap='RdBu_r',vmin=-20.,vmax=20.)
axes[2].set_title('CROCO-WRF Control WRF Latent HF ' + times[0] + ' ' + times[1])

# Shortwave heat flux:
(data['CROCO-WRF Smooth-SST']['day'].swrad.sel(time=slice(times[0],times[1])).mean('time')-data['CROCO-WRF Control']['day'].swrad.sel(time=slice(times[0],times[1])).mean('time')).plot(ax=axes[3],cmap='RdBu_r',vmin=-50.,vmax=50.)
axes[3].set_title('CROCO-WRF Control CROCO SW Radiation ' + times[0] + ' ' + times[1])

# Downward longwave heat flux:
SHFLX1 = (data['CROCO-WRF Control']['wrf_day'].GLW-5.67e-8*(data['CROCO-WRF Control']['wrf_day'].SST)**4).sel(time=slice(times[0],times[1])).mean('time')
SHFLX2 = (data['CROCO-WRF Smooth-SST']['wrf_day'].GLW-5.67e-8*(data['CROCO-WRF Smooth-SST']['wrf_day'].SST)**4).sel(time=slice(times[0],times[1])).mean('time')
(SHFLX2-SHFLX1).plot(ax=axes[4],cmap='RdBu_r',vmin=-20.,vmax=20.)
axes[4].set_title('CROCO-WRF Control WRF Longwave HF ' + times[0] + ' ' + times[1])

plt.tight_layout()

#plt.savefig('Smooth-SST/Surface_Fluxes_Example_2016-11-01_to_2016-11-31_Difference.png',dpi=100)

# Mean state differences:

In [ ]:
# Calculations
SST = {}
SSH = {}
SSS = {}
SWFLX = {}
TAUU = {}
TAUV = {}
U = {}
V = {}

keys = ['CROCO-WRF Control-E1','CROCO-WRF Control-E2','CROCO-WRF Control-E3','CROCO-WRF Control-E4','CROCO-WRF Control-E5',
        'CROCO-WRF Smooth-SST','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB']
for key in keys:
    print(key)
    SST[key] = data[key]['mon'].temp.isel(s_rho=-1).mean('time').load()
    SSH[key] = (data[key]['mon'].zeta.mean('time') - data[key]['mon'].zeta.mean('time').mean('x_rho').mean('y_rho')).load()
    SSS[key] = data[key]['mon'].salt.isel(s_rho=-1).mean('time').load()
    SWFLX[key] = data[key]['mon'].swflx.mean('time').load()
    TAUU[key] = data[key]['mon'].sustr.mean('time').load()
    TAUV[key] = data[key]['mon'].svstr.mean('time').load()
    U[key] = data[key]['mon'].u.isel(s_rho=-1).mean('time').load()
    V[key] = data[key]['mon'].v.isel(s_rho=-1).mean('time').load()

In [ ]:
# Calculate ensemble control:
for obj in [SST,SSH,SSS,SWFLX,TAUU,TAUV,U,V]:
    obj['CROCO-WRF Control'] = (obj['CROCO-WRF Control-E1']+obj['CROCO-WRF Control-E2']+obj['CROCO-WRF Control-E3']+obj['CROCO-WRF Control-E4']+obj['CROCO-WRF Control-E5'])/5.

In [ ]:
# Calculate significance levels for perturbations:
eL = 5
zScore = 2.58 # 1.96 = p-value 0.05, 2.58 = p-value 0.01 (two-sided)
for obj in [SST,SSH,SSS,SWFLX,TAUU,TAUV,U,V]:
    
    # Calculate standard deviation:
    std = np.zeros_like(obj['CROCO-WRF Control-E1'])
    for e in range(eL):
        std += (obj['CROCO-WRF Control-E%01d' % (e+1)]-obj['CROCO-WRF Control'])**2.
    std = np.sqrt(std/eL)
    
    for key in keys:
        obj[key + '-s'] = xr.zeros_like(obj[key])
        obj[key + '-s'].values = np.where(abs(obj[key].values-obj['CROCO-WRF Control'])>(zScore*std),1.,0.)
        
#     # Instead use outside range of ensemble members:
#     yL, xL = np.shape(obj['CROCO-WRF Control-E1'].values)
#     tmp = np.zeros((yL,xL,5))
#     for e in range(eL):
#         tmp[:,:,e] = obj[pert].values - obj['CROCO-WRF Control-E%01d' % (e+1)].values

#     obj[key + '-s'] = xr.zeros_like(obj['CROCO-WRF Control-E1'])
#     obj[key + '-s'].values = np.logical_or(np.all(tmp>0.,axis=2),np.all(tmp<0.,axis=2))

In [ ]:
fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(30,25),facecolor='w')
cont = 'CROCO-WRF Control'
hatch_sig = True

SST[cont].plot(ax=axes[0][0],vmin=20.,vmax=30.,cmap='RdBu_r')
axes[0][0].set_title(cont + ' SST [degC]')
SSH[cont].plot(ax=axes[0][1],vmin=-0.4,vmax=0.4,cmap='RdBu_r')
axes[0][1].set_title(cont + ' SSH [m]')
SSS[cont].plot(ax=axes[0][2],vmin=33.,vmax=36.,cmap='RdBu_r')
axes[0][2].set_title(cont + ' SSS [psu]')

keys = ['CROCO-WRF Smooth-SST','CROCO-WRF NoMesoCFB','CROCO-WRF NoCFB']
for i, key in enumerate(keys):
    (SST[key]-SST[cont]).plot(ax=axes[i+1][0],vmin=-0.25,vmax=0.25,cmap='RdBu_r',extend='both')
    if hatch_sig:  SST[key+'-s'].plot.contourf(ax=axes[i+1][0],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][0].set_title(key + ' SST difference [degC]')
    (SSH[key]-SSH[cont]).plot(ax=axes[i+1][1],vmin=-0.05,vmax=0.05,cmap='RdBu_r',extend='both')
    if hatch_sig:  SSH[key+'-s'].plot.contourf(ax=axes[i+1][1],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][1].set_title(key + ' SSH difference [m]')
    (SSS[key]-SSS[cont]).plot(ax=axes[i+1][2],vmin=-0.3,vmax=0.3,cmap='RdBu_r',extend='both')
    if hatch_sig:  SSS[key+'-s'].plot.contourf(ax=axes[i+1][2],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][2].set_title(key + ' SSS difference [psu]')
    
for axs in axes.flatten():
    axs.set_facecolor('k')

plt.tight_layout()
plt.savefig('Mean_State/CROCO-WRF_SST_SSH_SSS.png',dpi=100)

In [ ]:
fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(30,30),facecolor='w')
hatch_sig = True

U[cont].plot(ax=axes[0][0],vmin=-0.7,vmax=0.7,cmap='RdBu_r')
axes[0][0].set_title(cont + ' U [ms-1]')
V[cont].plot(ax=axes[0][1],vmin=-0.2,vmax=0.2,cmap='RdBu_r')
axes[0][1].set_title(cont + ' V [ms-1]')
(np.sqrt(grid.interp(V[cont],'y',boundary='fill').rename({'x_v':'x_rho'})**2.+grid.interp(U[cont],'x',boundary='fill').rename({'y_u':'y_rho'})**2.)).plot(ax=axes[0][2],vmin=0.,vmax=0.8,cmap=cm.cm.amp)
axes[0][2].set_title(cont + ' |U| [ms-1]')

keys = ['CROCO-WRF Smooth-SST','CROCO-WRF NoMesoCFB','CROCO-WRF NoCFB']
for i, key in enumerate(keys):
    (U[key]-U[cont]).plot(ax=axes[i+1][0],vmin=-0.15,vmax=0.15,cmap='RdBu_r',extend='both')
    if hatch_sig:  U[key+'-s'].plot.contourf(ax=axes[i+1][0],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][0].set_title(key + ' U  difference [ms-1]')
    (V[key]-V[cont]).plot(ax=axes[i+1][1],vmin=-0.1,vmax=0.1,cmap='RdBu_r',extend='both')
    if hatch_sig:  V[key+'-s'].plot.contourf(ax=axes[i+1][1],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][1].set_title(key + ' V  difference [ms-1]')
    (np.sqrt(grid.interp(V[key],'y',boundary='fill').rename({'x_v':'x_rho'})**2.+grid.interp(U[key],'x',boundary='fill').rename({'y_u':'y_rho'})**2.)-np.sqrt(grid.interp(V[cont],'y',boundary='fill').rename({'x_v':'x_rho'})**2.+grid.interp(U[cont],'x',boundary='fill').rename({'y_u':'y_rho'})**2.)).plot(ax=axes[i+1][2],vmin=-0.15,vmax=0.15,cmap='RdBu_r',extend='both')
    axes[i+1][2].set_title(key + ' |U|  difference [ms-1]')

for axs in axes.flatten():
    axs.set_facecolor('k')
plt.tight_layout()
plt.savefig('Mean_State/CROCO-WRF_U_V_Speed.png',dpi=100)

In [ ]:
fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(30,30),facecolor='w')
hatch_sig = True

SWFLX[cont].plot(ax=axes[0][0],vmin=-1.5,vmax=1.5,cmap='RdBu_r')
axes[0][0].set_title(cont + ' surface volume flux [cms-1]')
TAUU[cont].plot(ax=axes[0][1],vmin=-0.15,vmax=0.15,cmap='RdBu_r')
axes[0][1].set_title(cont + ' zonal wind stress [Nm-2]')
TAUV[cont].plot(ax=axes[0][2],vmin=-0.15,vmax=0.15,cmap='RdBu_r')
axes[0][2].set_title(cont + ' meridional wind stress [Nm-2]')

keys = ['CROCO-WRF Smooth-SST','CROCO-WRF NoMesoCFB','CROCO-WRF NoCFB']
for i, key in enumerate(keys):
    (SWFLX[key]-SWFLX[cont]).plot(ax=axes[i+1][0],vmin=-0.2,vmax=0.2,cmap='RdBu_r',extend='both')
    if hatch_sig:  SWFLX[key+'-s'].plot.contourf(ax=axes[i+1][0],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][0].set_title(key + ' surface volume flux difference [cms-1]')
    (TAUU[key]-TAUU[cont]).plot(ax=axes[i+1][1],vmin=-0.008,vmax=0.008,cmap='RdBu_r',extend='both')
    if hatch_sig:  TAUU[key+'-s'].plot.contourf(ax=axes[i+1][1],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][1].set_title(key + ' zonal wind stress difference [Nm-2]')
    (TAUV[key]-TAUV[cont]).plot(ax=axes[i+1][2],vmin=-0.008,vmax=0.008,cmap='RdBu_r',extend='both')
    if hatch_sig:  TAUV[key+'-s'].plot.contourf(ax=axes[i+1][2],levels=[-0.5, 0.5, 1.25],hatches=[None,"."],colors='none',add_colorbar=False)
    axes[i+1][2].set_title(key + ' meridional wind stress difference [Nm-2]')

for axs in axes.flatten():
    axs.set_facecolor('k')
plt.tight_layout()
plt.savefig('Mean_State/CROCO-WRF_P-E_TAU.png',dpi=100)

## Longitude surface profiles:

In [ ]:
LONS = 210., 260.

keys = ['CROCO-WRF Control',
        'CROCO-WRF Control-E1','CROCO-WRF Control-E2','CROCO-WRF Control-E3','CROCO-WRF Control-E4','CROCO-WRF Control-E5',
        'CROCO-WRF Smooth-SST','CROCO-WRF NoMesoCFB','CROCO-WRF NoCFB']
SSTll = {};SSHll = {};SWFLXll={};Ull={};Vll={};SSSll={};TAUUll={};TAUVll={};
for key in keys:
    print(key)
    SSTll[key] = SST[key].sel(x_rho=slice(LONS[0],LONS[1])).mean('x_rho').load()
    SSHll[key] = SSH[key].sel(x_rho=slice(LONS[0],LONS[1])).mean('x_rho').load()
    SWFLXll[key] = SWFLX[key].sel(x_rho=slice(LONS[0],LONS[1])).mean('x_rho').load()
    Ull[key] = U[key].sel(x_u=slice(LONS[0],LONS[1])).mean('x_u').load()
    Vll[key] = V[key].sel(x_v=slice(LONS[0],LONS[1])).mean('x_v').load()
    SSSll[key] = SSS[key].sel(x_rho=slice(LONS[0],LONS[1])).mean('x_rho').load()
    TAUUll[key] = TAUU[key].sel(x_u=slice(LONS[0],LONS[1])).mean('x_u').load()
    TAUVll[key] = TAUV[key].sel(x_v=slice(LONS[0],LONS[1])).mean('x_v').load()

In [ ]:
# Calculate significance levels for perturbations:
eL = 5
zScore = 2.58 # 1.96 = p-value 0.05, 2.58 = p-value 0.01 (two-sided)
for obj in [SSTll,SSHll,SSSll,SWFLXll,TAUUll,TAUVll,Ull,Vll]:
    
    # Calculate standard deviation:
    obj['CROCO-WRF Control-s'] = xr.zeros_like(obj['CROCO-WRF Control-E1'])
    for e in range(eL):
        obj['CROCO-WRF Control-s'] += (obj['CROCO-WRF Control-E%01d' % (e+1)]-obj['CROCO-WRF Control'])**2.
    obj['CROCO-WRF Control-s'] = np.sqrt(obj['CROCO-WRF Control-s']/eL)

In [ ]:
## Latitude line plots:
fig, axes = plt.subplots(nrows=1,ncols=5,figsize=(22,8),facecolor='w')

axs = axes.flatten()

anom = True
cont = 'CROCO-WRF Control'

keys = ['CROCO-WRF Control-E1','CROCO-WRF Control-E2','CROCO-WRF Control-E3','CROCO-WRF Control-E4','CROCO-WRF Control-E5',
        'CROCO-WRF Smooth-SST','CROCO-WRF NoMesoCFB','CROCO-WRF NoCFB']
cols = ['0.65','0.65','0.65','0.65','0.65',
        'C0','C1','C2']
labels = ['CROCO-WRF Control Ensemble','','','','',
          'CROCO-WRF Smooth-SST','CROCO-WRF No Mesoscale CFB','CROCO-WRF No CFB']
widths = [1.,1.,1.,1.,1.,
          3., 3., 3.]
for i,key in enumerate(keys):
    if anom:
        var = SSTll[key]-SSTll[cont]
    else:
        var = SSTll[key]
    if len(labels[i])>1:
        var.plot(y='y_rho',ax=axs[0],label=labels[i],color=cols[i],linewidth=widths[i])
    else:
        var.plot(y='y_rho',ax=axs[0],color=cols[i],linewidth=widths[i])
std = SSTll['CROCO-WRF Control-s']
if anom:
    axs[0].fill_betweenx(std.y_rho,-zScore*std,zScore*std,color='0.75',alpha=0.5)
else:
    axs[0].fill_betweenx(std.y_rho,-zScore*std+SSTll[cont],zScore*std+SSTll[cont],color='0.75',alpha=0.5)
axs[0].set_xlabel('SST [degC]')
axs[0].legend(loc='upper right')
if anom:
    axs[0].set_xlim([-0.11,0.1])
else:
    axs[0].set_xlim([26.,27.75])

for i,key in enumerate(keys):
    if anom:
        var = SSHll[key]-SSHll[cont]
    else:
        var = SSHll[key]
    var.plot(y='y_rho',ax=axs[1],color=cols[i],linewidth=widths[i])
std = SSHll['CROCO-WRF Control-s']
if anom:
    axs[1].fill_betweenx(std.y_rho,-zScore*std,zScore*std,color='0.75',alpha=0.5)
else:
    axs[1].fill_betweenx(std.y_rho,-zScore*std+SSHll[cont],zScore*std+SSHll[cont],color='0.75',alpha=0.5)
    
axs[1].set_xlabel('SSH [m]')
if anom:
    axs[1].set_xlim([-0.015,0.01])
else:
    axs[1].set_xlim([-0.12,0.])

for i,key in enumerate(keys):
    if anom:
        var = Ull[key]-Ull[cont]
    else:
        var = Ull[key]
    var.plot(y='y_u',ax=axs[2],color=cols[i],linewidth=widths[i])
std = Ull['CROCO-WRF Control-s']
if anom:
    axs[2].fill_betweenx(std.y_u,-zScore*std,zScore*std,color='0.75',alpha=0.5)
else:
    axs[2].fill_betweenx(std.y_u,-zScore*std+Ull[cont],zScore*std+Ull[cont],color='0.75',alpha=0.5)
axs[2].set_xlabel('Zonal Surface Velocity [ms-1]')
if anom:
    axs[2].set_xlim([-0.08,0.04])
else:
    axs[2].set_xlim([-0.5,0.1])


for i,key in enumerate(keys):
    if anom:
        var = (TAUUll[key]-TAUUll[cont])*1e3
    else:
        var = TAUUll[key]*1e3
    var.plot(y='y_u',ax=axs[3],color=cols[i],linewidth=widths[i])
std = TAUUll['CROCO-WRF Control-s']
if anom:
    axs[3].fill_betweenx(std.y_u,-zScore*std*1e3,zScore*std*1e3,color='0.75',alpha=0.5)
else:
    axs[3].fill_betweenx(std.y_u,(-zScore*std+TAUUll[cont])*1e3,(zScore*std+TAUUll[cont])*1e3,color='0.75',alpha=0.5)

axs[3].set_xlabel('Zonal Wind Stress [$10^{-3}$Nm-2]')
if anom:
    axs[3].set_xlim([-2,1])
else:
    axs[3].set_xlim([-60,35])

for i,key in enumerate(keys):
    if anom:
        var = (TAUVll[key]-TAUVll[cont])*1e3
    else:
        var = TAUVll[key]*1e3
    var.plot(y='y_v',ax=axs[4],color=cols[i],linewidth=widths[i])
std = TAUVll['CROCO-WRF Control-s']
if anom:
    axs[4].fill_betweenx(std.y_v,-zScore*std*1e3,zScore*std*1e3,color='0.75',alpha=0.5)
else:
    axs[4].fill_betweenx(std.y_v,(-zScore*std+TAUVll[cont])*1e3,(zScore*std+TAUVll[cont])*1e3,color='0.75',alpha=0.5)
axs[4].set_xlabel('Meridional Wind Stress [$10^{-3}$Nm-2]')
if anom:
    axs[4].set_xlim([-2.5,1.5])
else:
    axs[4].set_xlim([-30,20])

# for i,key in enumerate(keys):
#     if anom:
#         var = SSSll[key]-SSSll[cont]
#     else:
#         var = SSSll[key]
#     var.plot(y='y_rho',ax=axs[3],color=cols[i],linewidth=widths[i])
# std = SSSll['CROCO-WRF Control-s']
# axs[3].fill_betweenx(std.y_rho,-zScore*std,zScore*std,color='0.75',alpha=0.5)
# axs[3].set_title('%03dE-%03dE SSS [psu]' % (LONS[0],LONS[1]))
# if anom:
#     axs[3].set_xlim([-0.05,0.15])
# else:
#     axs[3].set_xlim([34.25,35.5])
    
for ax in axs:
    ax.grid()
    ax.set_ylim([-5.,10.])
    ax.set_ylabel('')
    ax.set_yticklabels({})
    ax.set_title('')
axs[0].set_ylabel('Latitude ($^\circ$N)')
axs[0].set_yticks(np.arange(-4.,12.,2.))
axs[0].set_yticklabels(np.arange(-4.,12.,2.))

plt.tight_layout()
plt.savefig('Mean_State/Latitude_Profile_210to260_CROCO-WRF_anom.png',dpi=100)

# SST, SSH and V variance (longitude filtering):

In [ ]:
# Get data:
SST = {}
SSH = {}
V = {}
U = {}
TAUX = {}
TAUY = {}
keys = ['CROCO-WRF Control-E1','CROCO-WRF Control-E2','CROCO-WRF Control-E3','CROCO-WRF Control-E4','CROCO-WRF Control-E5',
        'CROCO-WRF Smooth-SST','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB']
for key in keys:
    SST[key] = data[key]['day'].temp_surf
    SSH[key] = data[key]['day'].zeta
    V[key] = data[key]['day'].v_surf
    U[key] = data[key]['day'].u_surf
    TAUX[key] = data[key]['day'].sustr
    TAUY[key] = data[key]['day'].svstr
DX = (data[keys[0]]['day'].x_rho[1]-data[keys[0]]['day'].x_rho[0]).values

ln_hp_filt = 12.0

In [ ]:
# Limit data sets for speed:
lats = -2., 10.
lons = 210.-ln_hp_filt, 260.+ln_hp_filt
times = '2016-07-01','2018-07-01'

for key in keys:
    SST[key] = SST[key].sel(time=slice(times[0],times[1])).sel(y_rho=slice(lats[0],lats[1])).sel(x_rho=slice(lons[0],lons[1]))
    SSH[key] = SSH[key].sel(time=slice(times[0],times[1])).sel(y_rho=slice(lats[0],lats[1])).sel(x_rho=slice(lons[0],lons[1]))
    V[key] = V[key].sel(time=slice(times[0],times[1])).sel(y_v=slice(lats[0],lats[1])).sel(x_v=slice(lons[0],lons[1]))
    U[key] = U[key].sel(time=slice(times[0],times[1])).sel(y_u=slice(lats[0],lats[1])).sel(x_u=slice(lons[0],lons[1]))
    TAUX[key] =TAUX[key].sel(time=slice(times[0],times[1])).sel(y_u=slice(lats[0],lats[1])).sel(x_u=slice(lons[0],lons[1]))
    TAUY[key] =TAUY[key].sel(time=slice(times[0],times[1])).sel(y_v=slice(lats[0],lats[1])).sel(x_v=slice(lons[0],lons[1]))

In [ ]:
SST['CROCO-WRF Control-E1'].dims.find('x_')

In [ ]:
# Do high-pass filtering:
ln_hp_filt = 12.0
SST_hp_std = {}
for key in keys:
    print(key)
    SST_hp = (SST[key]-SST[key].rolling(x_rho=int(ln_hp_filt/DX),center=True).mean())
    SST_hp_std[key] = SST_hp.std('time')
    SST_hp_std[key].load()

In [ ]:
SSH_hp_std = {}
for key in keys:
    print(key)
    SSH_hp = (SSH[key]-SSH[key].rolling(x_rho=int(ln_hp_filt/DX),center=True).mean())
    SSH_hp_std[key] = SSH_hp.std('time')
    SSH_hp_std[key].load()

In [ ]:
V_hp_std = {}
V_lp_std = {}
EWWV = {}
MWWV = {}
for key in keys:
    print(key)
    V_lp = V[key].rolling(x_v=int(ln_hp_filt/DX),center=True).mean()
    V_hp = V[key]-V_lp
    
    TAUY_lp = TAUY[key].rolling(x_v=int(ln_hp_filt/DX),center=True).mean()
    TAUY_hp = TAUY[key]-TAUY_lp
    
    V_hp_std[key] = V_hp.std('time').load()
    V_lp_std[key] = V_lp.std('time').load()
    
    EWWV[key] = (V_hp*TAUY_hp).mean('time').load()
    MWWV[key] = (V_lp*TAUY_lp).mean('time').load()

In [ ]:
U_hp_std = {}
U_lp_std = {}
EWWU = {}
MWWU = {}
for key in keys:
    print(key)
    U_lp = U[key].rolling(x_u=int(ln_hp_filt/DX),center=True).mean()
    U_hp = (U[key]-U_lp)
    
    TAUX_lp = TAUX[key].rolling(x_u=int(ln_hp_filt/DX),center=True).mean()
    TAUX_hp = TAUX[key]-TAUX_lp
    
    U_hp_std[key] = U_hp.std('time').load()
    U_lp_std[key] = U_lp.std('time').load()
    
    EWWU[key] = (U_hp*TAUX_hp).mean('time').load()
    MWWU[key] = (U_lp*TAUX_lp).mean('time').load()

In [ ]:
sq = 2.
tlims = 0.,1.**sq
zlims = 0.,0.08**sq
vlims = 0.,0.4**sq
tsp = tlims[1]/20.
zsp = zlims[1]/20.
vsp = vlims[1]/20.
reg = -160.+360.,-100.+360.
regy = -2.,6.

keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF NoMesoCFB-nomask','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-nomask']
# Do the plotting:
fig, axes = plt.subplots(nrows=6,ncols=4,figsize=(35,32),facecolor='w')
for i,key in enumerate(keys):
    (SST_hp_std[key]**sq).plot.contourf(ax=axes[i][0],levels=np.arange(tlims[0],tlims[1]+tsp,tsp),cmap=cm.cm.amp)
    (SSH_hp_std[key]**sq).plot.contourf(ax=axes[i][1],levels=np.arange(zlims[0],zlims[1]+zsp,zsp),cmap=cm.cm.amp)
    (V_hp_std[key]**sq).plot.contourf(ax=axes[i][2],levels=np.arange(vlims[0],vlims[1]+vsp,vsp),cmap=cm.cm.amp)
    (U_hp_std[key]**sq).plot.contourf(ax=axes[i][3],levels=np.arange(vlims[0],vlims[1]+vsp,vsp),cmap=cm.cm.amp)
    if sq>1.5:
        axes[i][0].set_title(key + ' SST variance ($^\circ$C$^2$)')
        axes[i][1].set_title(key + ' SSH variance (m2)')
        axes[i][2].set_title(key + ' V variance (m2s-2)')
        axes[i][3].set_title(key + ' U svariance(m2s-2)')
    else:
        axes[i][0].set_title(key + ' SST standard deviation ($^\circ$C)')
        axes[i][1].set_title(key + ' SSH standard deviation (m)')
        axes[i][2].set_title(key + ' V standard deviation (ms-1)')
        axes[i][3].set_title(key + ' U standard deviation (ms-1)')

axes[0][0].plot([reg[0],reg[0]],lats,'--k')
axes[0][0].plot([reg[1],reg[1]],lats,'--k') 
axes[0][0].plot([reg[0],reg[1]],[regy[0],regy[0]],'--k')
axes[0][0].plot([reg[0],reg[1]],[regy[1],regy[1]],'--k')   

for ax in axes.reshape(-1):
    ax.set_xlim([lons[0]+ln_hp_filt/2.,lons[1]-ln_hp_filt/2.])
    ax.grid()
plt.tight_layout()
plt.savefig('CFB_Figures/SSH_SST_V_U_HighPassLon_Variance_2016-07_2017-02_CROCO-WRF_All.png',dpi=150)

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(25,8))

sq = 2.

SSTv = {}
SSHv = {}
Uv = {}
Vv = {}
keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF NoMesoCFB-nomask','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-nomask']
for key in keys:
    (SST_hp_std[key]**sq).sel(x_rho=slice(reg[0],reg[1])).mean('x_rho').plot(ax=axes[0],label=key)
    (SSH_hp_std[key]**sq).sel(x_rho=slice(reg[0],reg[1])).mean('x_rho').plot(ax=axes[1],label=key)
    (V_hp_std[key]**sq).sel(x_v=slice(reg[0],reg[1])).mean('x_v').plot(ax=axes[2],label=key)
    (U_hp_std[key]**sq).sel(x_u=slice(reg[0],reg[1])).mean('x_u').plot(ax=axes[3],label=key)
    
    SSTv[key] = (SST_hp_std[key]**sq).sel(x_rho=slice(reg[0],reg[1])).sel(y_rho=slice(regy[0],regy[1])).mean('x_rho').mean('y_rho')    
    SSHv[key] = (SSH_hp_std[key]**sq).sel(x_rho=slice(reg[0],reg[1])).sel(y_rho=slice(regy[0],regy[1])).mean('x_rho').mean('y_rho')    
    Uv[key] = (U_hp_std[key]**sq).sel(x_u=slice(reg[0],reg[1])).sel(y_u=slice(regy[0],regy[1])).mean('x_u').mean('y_u')    
    Vv[key] = (V_hp_std[key]**sq).sel(x_v=slice(reg[0],reg[1])).sel(y_v=slice(regy[0],regy[1])).mean('x_v').mean('y_v')

print('SST variances:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (SSTv[key].values,SSTv[key].values/SSTv[cont].values*100))
print('SSH variances:')
for key in keys:
    print(key + ' = %5.5f (%02d)' % (SSHv[key].values,SSHv[key].values/SSHv[cont].values*100))
print('U variances:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (Vv[key].values,Uv[key].values/Uv[cont].values*100))
print('V variances:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (Vv[key].values,Vv[key].values/Vv[cont].values*100))
            
for ax in axes:
    for obj in ax.lines:   
        newx = obj.get_ydata()
        newy = obj.get_xdata()

# set new x- and y- data for the line
        obj.set_xdata(newx)
        obj.set_ydata(newy)

for axs in axes.reshape(-1):
    axs.set_ylim([-2.,10.])
    axs.grid()
    axs.set_ylabel('Latitude (degN)')
axes[0].legend()
if sq>1.5:
    axes[1].set_xlim([0.,0.0025])
    axes[2].set_xlim([0.,0.14])
    axes[3].set_xlim([0.,0.1])
    axes[0].set_xlim([0.,1.0])
    axes[0].set_xlabel('150W-90W SST variance ($^\circ$C$^2$)')
    axes[1].set_xlabel('150W-90W SSH variance (m2)')
    axes[2].set_xlabel('150W-90W V variance (m2s-2)')
    axes[3].set_xlabel('150W-90W U variance (m2s-2)')
else:
    axes[0].set_xlim([0.,1.])
    axes[1].set_xlim([0.,0.05])
    axes[2].set_xlim([0.,0.4])
    axes[3].set_xlim([0.,0.4])
    axes[0].set_xlabel('150W-90W SST standard deviatio ($^\circ$C)')
    axes[1].set_xlabel('150W-90W SSH standard deviatio (m)')
    axes[2].set_xlabel('150W-90W V standard deviatio (ms-1)')
    axes[3].set_xlabel('150W-90W U standard deviatio (ms-1)')

plt.savefig('SSH_SST_V_U_HighPassLon_Variance_2016-07_2017-02_CROCO-WRF_All_lonavg.png',dpi=150)

In [ ]:
# Percentage change:
fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(25,8))

cont = 'CROCO-WRF Control'
ControlSST = SST_hp_std[cont].sel(x_rho=slice(reg[0],reg[1])).mean('x_rho')
ControlSSH = SSH_hp_std[cont].sel(x_rho=slice(reg[0],reg[1])).mean('x_rho')
ControlV = V_hp_std[cont].sel(x_v=slice(reg[0],reg[1]+360.)).mean('x_v')
ControlU = U_hp_std[cont].sel(x_u=slice(reg[0],reg[1]+360.)).mean('x_u')

keys = ['CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF NoMesoCFB-nomask','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-nomask']
for key in keys:
    PertSST = SST_hp_std[key].sel(x_rho=slice(reg[0],reg[1])).mean('x_rho')
    PertSSH = SSH_hp_std[key].sel(x_rho=slice(reg[0],reg[1])).mean('x_rho')
    PertV = V_hp_std[key].sel(x_v=slice(reg[0],reg[1])).mean('x_v')
    PertU = U_hp_std[key].sel(x_u=slice(reg[0],reg[1])).mean('x_u')


    ((PertSST/ControlSST-1.)*100.).plot(ax=axes[0],label=key)
    ((PertSSH/ControlSSH-1.)*100.).plot(ax=axes[1],label=key)
    ((PertV/ControlV-1.)*100.).plot(ax=axes[2],label=key)
    ((PertU/ControlU-1.)*100.).plot(ax=axes[3],label=key)

for ax in axes:
    for obj in ax.lines:   
        newx = obj.get_ydata()
        newy = obj.get_xdata()

# set new x- and y- data for the line
        obj.set_xdata(newx)
        obj.set_ydata(newy)

for axs in axes.reshape(-1):
    axs.set_ylim([-2.,10.])
    axs.set_xlim([-100.,100.])
    axs.grid()
    axs.legend()
    axs.set_ylabel('Latitude (degN)')
axes[0].set_xlabel('150W-90W SST standard deviation %-change')
axes[1].set_xlabel('150W-90W SSH standard deviation %-change')
axes[2].set_xlabel('150W-90W V standard deviation %-change')
axes[3].set_xlabel('150W-90W U standard deviation %-change')

plt.savefig('CFB_Figures/SSH_SST_V_U_HighPassLon_2016-07_2017-02_CROCO-WRF_All_lonavg_pertchange.png',dpi=150)

# EWW, MWW, MKE, EKE:

In [ ]:
keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF NoMesoCFB-nomask','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-nomask']
# Do the plotting:
fig, axes = plt.subplots(nrows=6,ncols=4,figsize=(35,32),facecolor='w')
for i,key in enumerate(keys):
    EWW = EWWV[key]+EWWU[key].values
    MWW = MWWV[key]+MWWU[key].values
    EKE = (V_hp_std[key]**2.+U_hp_std[key].values**2.)
    MKE = (V_lp_std[key]**2.+U_lp_std[key].values**2.)
    
    (EWW).plot(ax=axes[i][0],vmin=-0.003,vmax=0.003,cmap='RdBu_r')
    (MWW).plot(ax=axes[i][1],vmin=-0.04,vmax=0.04,cmap='RdBu_r')
    (EKE).plot(ax=axes[i][2],vmin=0.,vmax=0.3,cmap=cm.cm.amp)
    (MKE).plot(ax=axes[i][3],vmin=0.,vmax=0.15,cmap=cm.cm.amp)
    
    axes[i][0].set_title(key + ' EWW [Nms-1]')
    axes[i][1].set_title(key + ' MWW [Nms-1]')
    axes[i][2].set_title(key + ' EKE [m2s-2]')
    axes[i][3].set_title(key + ' MKE [m2s-2]')

for ax in axes.reshape(-1):
    ax.set_xlim([lons[0]+ln_hp_filt/2.,lons[1]-ln_hp_filt/2.])
    ax.grid()
plt.tight_layout()
plt.savefig('CFB_Figures/WindWork_MKE_EKE_2016-07_2017-02_All.png',dpi=150)

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=4,figsize=(25,8))

sq = 2.
reg = -160.+360.,-100.+360.
regy = -2.,6.

EKEv = {}
MKEv = {}
EWWv = {}
MWWv = {}
keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF NoMesoCFB-nomask','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-nomask']
for key in keys:
    
    EWW = EWWV[key]+EWWU[key].values
    MWW = MWWV[key]+MWWU[key].values
    EKE = (V_hp_std[key]**2.+U_hp_std[key].values**2.)
    MKE = (V_lp_std[key]**2.+U_lp_std[key].values**2.)
    
    EWW.sel(x_v=slice(reg[0],reg[1])).mean('x_v').plot(ax=axes[0],label=key)
    MWW.sel(x_v=slice(reg[0],reg[1])).mean('x_v').plot(ax=axes[1],label=key)
    EKE.sel(x_v=slice(reg[0],reg[1])).mean('x_v').plot(ax=axes[2],label=key)
    MKE.sel(x_v=slice(reg[0],reg[1])).mean('x_v').plot(ax=axes[3],label=key)
    
    EWWv[key] = EWW.sel(x_v=slice(reg[0],reg[1])).sel(y_v=slice(regy[0],regy[1])).mean('x_v').mean('y_v')    
    MWWv[key] = MWW.sel(x_v=slice(reg[0],reg[1])).sel(y_v=slice(regy[0],regy[1])).mean('x_v').mean('y_v')    
    EKEv[key] = EKE.sel(x_v=slice(reg[0],reg[1])).sel(y_v=slice(regy[0],regy[1])).mean('x_v').mean('y_v')    
    MKEv[key] = MKE.sel(x_v=slice(reg[0],reg[1])).sel(y_v=slice(regy[0],regy[1])).mean('x_v').mean('y_v')

print('EWW:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (EWWv[key].values,EWWv[key].values/EWWv[cont].values*100))
print('MWW:')
for key in keys:
    print(key + ' = %5.5f (%02d)' % (MWWv[key].values,MWWv[key].values/MWWv[cont].values*100))
print('EKE:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (EKEv[key].values,EKEv[key].values/EKEv[cont].values*100))
print('MKE:')
for key in keys:
    print(key + ' = %4.3f (%02d)' % (MKEv[key].values,MKEv[key].values/MKEv[cont].values*100))
            
for ax in axes:
    for obj in ax.lines:   
        newx = obj.get_ydata()
        newy = obj.get_xdata()

# set new x- and y- data for the line
        obj.set_xdata(newx)
        obj.set_ydata(newy)

for axs in axes.reshape(-1):
    axs.set_ylim([-2.,10.])
    
    axs.grid()
    axs.set_ylabel('Latitude (degN)')
axes[2].legend()
axes[0].set_xlim([-0.002,0.001])
axes[1].set_xlim([-0.01,0.04])
axes[2].set_xlim([0.,0.2])
axes[3].set_xlim([0.,0.1])
axes[0].set_xlabel('150W-90W EWW')
axes[1].set_xlabel('150W-90W MWW')
axes[2].set_xlabel('150W-90W EKE')
axes[3].set_xlabel('150W-90W MKE')

plt.savefig('CFB_Figures/WindWork_MKE_EKE_2016-07_2017-02_lonavg_All.png',dpi=150)

# EKE, MKE and wind works:

In [ ]:
# Get data:
V = {}
U = {}
TAUX = {}
TAUY = {}
keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF Smooth-SST']

for key in keys:
    V[key] = data[key]['day'].v_surf
    U[key] = data[key]['day'].u_surf
    TAUX[key] = data[key]['day'].sustr
    TAUY[key] = data[key]['day'].svstr
DX = (data[keys[0]]['day'].x_rho[1]-data[keys[0]]['day'].x_rho[0]).values

ln_hp_filt = 12.0

In [ ]:
# Limit data sets for speed:
lats = -2., 10.
lons = 180., 275.
#times = '2016-07-01','2018-07-01'
times = '2016-07-01','2017-02-01'

for key in keys:
    V[key] = V[key].sel(time=slice(times[0],times[1])).sel(y_v=slice(lats[0],lats[1])).sel(x_v=slice(lons[0],lons[1]))
    U[key] = U[key].sel(time=slice(times[0],times[1])).sel(y_u=slice(lats[0],lats[1])).sel(x_u=slice(lons[0],lons[1]))
    TAUX[key] =TAUX[key].sel(time=slice(times[0],times[1])).sel(y_u=slice(lats[0],lats[1])).sel(x_u=slice(lons[0],lons[1]))
    TAUY[key] =TAUY[key].sel(time=slice(times[0],times[1])).sel(y_v=slice(lats[0],lats[1])).sel(x_v=slice(lons[0],lons[1]))

In [ ]:
# Do (delayed) calculations of pure fields:
EKEU = {}
EKEV = {}
MKEU = {}
MKEV = {}
EWWU = {}
EWWV = {}
MWWU = {}
MWWV = {}

for key in keys:
    V_lp = V[key].rolling(x_v=int(ln_hp_filt/DX),center=True).mean()
    V_hp = V[key]-V_lp
    U_lp = U[key].rolling(x_u=int(ln_hp_filt/DX),center=True).mean()
    U_hp = U[key]-U_lp
    
    TAUY_lp = TAUY[key].rolling(x_v=int(ln_hp_filt/DX),center=True).mean()
    TAUY_hp = TAUY[key]-TAUY_lp
    TAUX_lp = TAUX[key].rolling(x_u=int(ln_hp_filt/DX),center=True).mean()
    TAUX_hp = TAUX[key]-TAUX_lp
    
    EKEV[key] = V_hp**2.
    MKEV[key] = V_lp**2.
    EKEU[key] = U_hp**2.
    MKEU[key] = U_lp**2.
    
    EWWV[key] = V_hp*TAUY_hp
    EWWU[key] = U_hp*TAUX_hp
    MWWV[key] = V_lp*TAUY_lp
    MWWU[key] = U_lp*TAUX_lp

In [ ]:
# Actually do calculations, by time:
# Split calculation into single time-step chunks:
EKEU_sp = {}
EKEV_sp = {}
MKEU_sp = {}
MKEV_sp = {}

EWWU_sp = {}
EWWV_sp = {}
MWWU_sp = {}
MWWV_sp = {}

EKEU_ts = {}
EKEV_ts = {}
MKEU_ts = {}
MKEV_ts = {}

EWWU_ts = {}
EWWV_ts = {}
MWWU_ts = {}
MWWV_ts = {}
for key in keys:
    EKEU_sp[key] = xr.zeros_like(EKEU[key].isel(time=0))
    EKEV_sp[key] = xr.zeros_like(EKEV[key].isel(time=0))
    MKEU_sp[key] = xr.zeros_like(MKEU[key].isel(time=0))
    MKEV_sp[key] = xr.zeros_like(MKEV[key].isel(time=0))

    EWWU_sp[key] = xr.zeros_like(EWWU[key].isel(time=0))
    EWWV_sp[key] = xr.zeros_like(EWWV[key].isel(time=0))
    MWWU_sp[key] = xr.zeros_like(MWWU[key].isel(time=0))
    MWWV_sp[key] = xr.zeros_like(MWWV[key].isel(time=0))

    EKEU_ts[key] = xr.zeros_like(EKEU[key].isel(x_u=0).isel(y_u=0))
    EKEV_ts[key] = xr.zeros_like(EKEV[key].isel(x_v=0).isel(y_v=0))
    MKEU_ts[key] = xr.zeros_like(MKEU[key].isel(x_u=0).isel(y_u=0))
    MKEV_ts[key] = xr.zeros_like(MKEV[key].isel(x_v=0).isel(y_v=0))

    EWWU_ts[key] = xr.zeros_like(EWWU[key].isel(x_u=0).isel(y_u=0))
    EWWV_ts[key] = xr.zeros_like(EWWV[key].isel(x_v=0).isel(y_v=0))
    MWWU_ts[key] = xr.zeros_like(MWWU[key].isel(x_u=0).isel(y_u=0))
    MWWV_ts[key] = xr.zeros_like(MWWV[key].isel(x_v=0).isel(y_v=0))
    tL = len(EKEU[key].time)
    
    for ii in range(tL):
        print('Doing time step ' + str(ii+1) + ' of ' + str(tL) + ' Run ' + key)
        EKEU_t = EKEU[key].isel(time=ii).load()
        EKEV_t = EKEV[key].isel(time=ii).load()
        MKEU_t = MKEU[key].isel(time=ii).load()
        MKEV_t = MKEV[key].isel(time=ii).load()
        EWWU_t = EWWU[key].isel(time=ii).load()
        EWWV_t = EWWV[key].isel(time=ii).load()
        MWWU_t = MWWU[key].isel(time=ii).load()
        MWWV_t = MWWV[key].isel(time=ii).load()
        
        EKEU_sp[key] = EKEU_sp[key]+EKEU_t/tL
        EKEV_sp[key] = EKEV_sp[key]+EKEV_t/tL    
        MKEU_sp[key] = MKEU_sp[key]+MKEU_t/tL
        MKEV_sp[key] = MKEV_sp[key]+MKEV_t/tL
        EKEU_ts[key][ii] = EKEU_t.mean('x_u').mean('y_u')
        EKEV_ts[key][ii] = EKEV_t.mean('x_v').mean('y_v')    
        MKEU_ts[key][ii] = MKEU_t.mean('x_u').mean('y_u')
        MKEV_ts[key][ii] = MKEV_t.mean('x_v').mean('y_v')   
        
        EWWU_sp[key] = EWWU_sp[key]+EWWU_t/tL
        EWWV_sp[key] = EWWV_sp[key]+EWWV_t/tL    
        MWWU_sp[key] = MWWU_sp[key]+MWWU_t/tL
        MWWV_sp[key] = MWWV_sp[key]+MWWV_t/tL
        EWWU_ts[key][ii] = EWWU_t.mean('x_u').mean('y_u')
        EWWV_ts[key][ii] = EWWV_t.mean('x_v').mean('y_v')    
        MWWU_ts[key][ii] = MWWU_t.mean('x_u').mean('y_u')
        MWWV_ts[key][ii] = MWWV_t.mean('x_v').mean('y_v') 

In [ ]:
tlims = 0.,1.
zlims = 0.,1.
vlims = 0.,0.1
clims = 0.,0.1
tsp = tlims[1]/20.
zsp = zlims[1]/20.
vsp = vlims[1]/20.
csp = clims[1]/20.

keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF Smooth-SST']
# Do the plotting:
fig, axes = plt.subplots(nrows=4,ncols=4,figsize=(35,24),facecolor='w')
for i,key in enumerate(keys):
#    (grid.interp(EKEU_sp[key],'x',boundary='fill').rename({'x_u':'x_rho','y_u':'y_rho'})+grid.interp(EKEV_sp[key],'y',boundary='fill').rename({'x_v':'x_rho','y_v':'y_rho'})).plot.contourf(ax=axes[i][0],levels=np.arange(tlims[0],tlims[1]+tsp,tsp),cmap=cm.cm.amp)
#    (grid.interp(MKEU_sp[key],'x',boundary='fill').rename({'x_u':'x_rho','y_u':'y_rho'})+grid.interp(MKEV_sp[key],'y',boundary='fill').rename({'x_v':'x_rho','y_v':'y_rho'})).plot.contourf(ax=axes[i][1],levels=np.arange(zlims[0],zlims[1]+zsp,zsp),cmap=cm.cm.amp)
#    (grid.interp(EWWU_sp[key],'x',boundary='fill').rename({'x_u':'x_rho','y_u':'y_rho'})+grid.interp(EWWV_sp[key],'y',boundary='fill').rename({'x_v':'x_rho','y_v':'y_rho'})).plot.contourf(ax=axes[i][2],levels=np.arange(vlims[0],vlims[1]+vsp,vsp),cmap=cm.cm.amp)
#    (grid.interp(MWWU_sp[key],'x',boundary='fill').rename({'x_u':'x_rho','y_u':'y_rho'})+grid.interp(MWWV_sp[key],'y',boundary='fill').rename({'x_v':'x_rho','y_v':'y_rho'})).plot.contourf(ax=axes[i][3],levels=np.arange(vlims[0],clims[1]+csp,csp),cmap=cm.cm.amp)
    (EKEU_sp[key]).plot.contourf(ax=axes[i][0],levels=np.arange(tlims[0],tlims[1]+tsp,tsp),cmap=cm.cm.amp)
    (MKEU_sp[key]).plot.contourf(ax=axes[i][1],levels=np.arange(zlims[0],zlims[1]+zsp,zsp),cmap=cm.cm.amp)
    (EWWU_sp[key]).plot.contourf(ax=axes[i][2],levels=np.arange(vlims[0],vlims[1]+vsp,vsp),cmap=cm.cm.amp)
    (MWWU_sp[key]).plot.contourf(ax=axes[i][3],levels=np.arange(vlims[0],clims[1]+csp,csp),cmap=cm.cm.amp)
    
   
    axes[i][0].set_title(key + ' EKE (m2s-2)')
    axes[i][1].set_title(key + ' MKE (m2s-2)')
    axes[i][2].set_title(key + ' EWW (Nms-1)')
    axes[i][3].set_title(key + ' MWW (Nms-1)')

for ax in axes.reshape(-1):
    ax.set_xlim([lons[0]+ln_hp_filt/2.,lons[1]-ln_hp_filt/2.])
    ax.grid()
plt.tight_layout()
#plt.savefig('CFB_Figures/SSH_SST_V_U_HighPassLon_Variance_2016-07_2018-06_CROCO-WRF_CFB.png',dpi=150)

In [ ]:
fig, axes = plt.subplots(nrows=4,ncols=1,figsize=(15,20),facecolor='w')
keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF Smooth-SST']
# Do the plotting:
for i,key in enumerate(keys):
    (EKEU_ts[key]+EKEV_ts[key]).plot(ax=axes[0],label=key)
    (MKEU_ts[key]+MKEV_ts[key]).plot(ax=axes[1],label=key)
    (EWWU_ts[key]+EWWV_ts[key]).plot(ax=axes[2],label=key)
    (MWWU_ts[key]+MWWV_ts[key]).plot(ax=axes[3],label=key)
    
axes[0].set_title('EKE (m2s-2)')
axes[0].legend()
axes[1].set_title('MKE (m2s-2)')
axes[2].set_title('EWW (Nms-1)')
axes[3].set_title('MWW (Nms-1)')

for ax in axes.reshape(-1):
    ax.grid()
plt.tight_layout()
#plt.savefig('CFB_Figures/SSH_SST_V_U_HighPassLon_Variance_2016-07_2018-06_CROCO-WRF_CFB.png',dpi=150)

In [ ]:
# Wind work and KEs (time series and spatial plots):
EKEU = {}
EKEV = {}
MKEU = {}
MKEV = {}
EWWU = {}
EWWV = {}
MWWU = {}
EWWU = {}


keys = ['CROCO-WRF Control','CROCO-WRF NoCFB','CROCO-WRF NoMesoCFB','CROCO-WRF Smooth-SST']
for key in keys:
    print(key)
    U = data[key]['day'].u_surf
    V = data[key]['day'].v_surf
    TAUX = data[key]['day'].sustr
    TAUY = data[key]['day'].svstr

# Some snapshots and TIW filtering (WRF data):

In [ ]:
#Calculations:
time = '2016-11-01'

ln_hp_filt = 12.0
DX = (data['CROCO-WRF Control']['wrf_day'].nav_lon[0,1]-data['CROCO-WRF Control']['wrf_day'].nav_lon[0,0]).values

SST = (data['CROCO-WRF Control']['wrf_day'].SST-273.15).sel(time=time)
SST = SST.where(SST>-10.)
WSP = data['CROCO-WRF Control']['wrf_day'].WSPD10.sel(time=time)
LH = data['CROCO-WRF Control']['wrf_day'].LH.sel(time=time)

SST_TIW = (SST-SST.rolling(x=int(ln_hp_filt/DX),center=True).mean())
WSP_TIW = (WSP-WSP.rolling(x=int(ln_hp_filt/DX),center=True).mean())

In [ ]:
fig,axes = plt.subplots(nrows=3,ncols=1,figsize=(16,15))

SST.isel(time=0).plot.contour(levels=np.arange(0.,30.,0.5),colors='k',ax=axes[0])
SST_TIW.plot(ax=axes[0],vmin=-2.,vmax=2.,cmap='RdBu_r')
#WSP_TIW.isel(time=0).plot.contour(levels=np.arange(-3.0,0.,0.3),linestyle='dashed',colors='b',ax=axes)
#WSP_TIW.isel(time=0).plot.contour(levels=np.arange(0.3,3.3,0.3),colors='b',ax=axes)

WSP_TIW.plot(ax=axes[1],vmin=-1.5,vmax=1.5,cmap='RdBu_r')
SST_TIW.isel(time=0).plot.contour(levels=np.arange(-3.0,0.,0.5),linestyle='dashed',colors='k',ax=axes[1])
SST_TIW.isel(time=0).plot.contour(levels=np.arange(0.5,3.5,0.5),colors='k',ax=axes[1])

SST.isel(time=0).plot.contour(levels=np.arange(0.,30.,0.5),colors='k',ax=axes[2])
(-LH).plot(ax=axes[2],vmin=-200.,vmax=200.,cmap='RdBu_r')

for ax in axes:
    ax.set_xlim([180.,280.])
    ax.set_ylim([-6.,9.])
    ax.set_xlabel('Longitude ($^\circ$E)')
    ax.set_ylabel('Latitude ($^\circ$N)')
axes[0].set_title(time + ' SST and TIW SST anomalies (degC)')
axes[1].set_title(time + ' TIW 10m wind speed anomalies (ms-1) and SST anomaly contours (0.5degC)')
axes[2].set_title(time + ' Latent Heat Flux (Wm-2) and SST contours (0.5degC)')
plt.tight_layout()
plt.savefig('SST_WSP_LH.png',dpi=200)

# Analysis/comparison of 2016-2017 La Nina:

In [ ]:
# Region definitions:

TIWyreg = [-5.,8.]
TIWxreg = [200.,260.]
N34yreg = [-5.,5.]
#N34xreg = [-170.+360.,-120.+360.]
#N34xreg = [-150.+360.,-90.+360.]
N34xreg = TIWxreg

In [ ]:
# Monthly SSTs:
fig,axes = plt.subplots(nrows=3,ncols=2,figsize=(20,15),facecolor='w')
axs = axes.reshape(-1)

SST_CONT = data['CROCO-WRF Control']['mon'].temp.isel(s_rho=-1)
SST_SMTH = data['CROCO-WRF Smooth-SST-E2']['mon'].temp.isel(s_rho=-1)

month_one = '2016-09-15'
cont_ind = list(SST_CONT.time.values).index(SST_CONT.sel(time=month_one, method='nearest').time)
smth_ind = list(SST_SMTH.time.values).index(SST_SMTH.sel(time=month_one, method='nearest').time)

for i in range(6):
    print(i)
    SST1 = SST_CONT.isel(time=i+cont_ind)
    SST2 = SST_SMTH.isel(time=i+smth_ind)
    (SST2-SST1).plot(ax=axs[i],vmin=-3.,vmax=3.,cmap='RdBu_r')
    axs[i].set_title('CROCO-WRF Smooth-SST - CROCO-WRF Control SST ' + str(SST1.time.values)[:10])
i = 2
axs[i].add_patch(Rectangle((TIWxreg[0], TIWyreg[0]), TIWxreg[1]-TIWxreg[0], TIWyreg[1]-TIWyreg[0],
             edgecolor = 'black',
             fill=False,
             lw=1))
    
axs[i].add_patch(Rectangle((N34xreg[0], N34yreg[0]), N34xreg[1]-N34xreg[0], N34yreg[1]-N34yreg[0],
             edgecolor = 'red',
             fill=False,
             lw=1))

plt.tight_layout()
#plt.gca().set_xlim([180.,280.])
#plt.gca().set_ylim([-6.,9.])
plt.savefig('Smooth-SST/LaNina_2016-2017_Monthly_SST_Difference-E2.png',dpi=100)

In [ ]:
#Calculations:
ln_hp_filt = 12.0
keys = ['CROCO-WRF Control','CROCO-WRF Smooth-SST','CROCO-WRF Smooth-SST-E2']

SST = {}
SST_TIW = {}
V_TIW = {}
DX = (data['CROCO-WRF Control']['day'].x_rho[1]-data['CROCO-WRF Control']['day'].x_rho[0]).values

for key in keys:
    SST[key] = data[key]['day'].temp_surf
    SST[key] = SST[key].where(SST[key]>-10.)
    SST_TIW[key] = (SST[key]-SST[key].rolling(x_rho=int(ln_hp_filt/DX),center=True).mean())
    
    V = data[key]['day'].v_surf
    V_TIW[key] = (V-V.rolling(x_v=int(ln_hp_filt/DX),center=True).mean())

In [ ]:
times = ['2016-09-01','2017-03-01']
for key in keys:
    print(key + ' SST')
    SST[key] = SST[key].sel(time=slice(times[0],times[1])).load()
    SST_TIW[key] = SST_TIW[key].sel(time=slice(times[0],times[1])).load()
    print(key + ' V')
    V_TIW[key] = V_TIW[key].sel(time=slice(times[0],times[1])).load()


In [ ]:
time = '2016-11-01'
fig,axes = plt.subplots(nrows=2,ncols=2,figsize=(16,13),facecolor='w')

for i, key in enumerate(keys):
    SST_TIW[key].sel(time=time).plot(ax=axes[i][0],vmin=-2.,vmax=2.,cmap='RdBu_r')
    axes[i][0].set_title('TIW-SST ' + key)
    
    V_TIW[key].sel(time=time).plot(ax=axes[i][1],vmin=-1.5,vmax=1.5,cmap='RdBu_r')
    axes[i][1].set_title('TIW-V ' + key)

    axes[i][1].add_patch(Rectangle((TIWxreg[0], TIWyreg[0]), TIWxreg[1]-TIWxreg[0], TIWyreg[1]-TIWyreg[0],
             edgecolor = 'black',
             fill=False,
             lw=1))
    
    axes[i][1].add_patch(Rectangle((N34xreg[0], N34yreg[0]), N34xreg[1]-N34xreg[0], N34yreg[1]-N34yreg[0],
             edgecolor = 'red',
             fill=False,
             lw=1))
    
plt.savefig('Smooth-SST/LaNina_2016-2017_TIW_V-SST.png',dpi=100)

In [ ]:
yflx = 3.

SST_TIW_ts = {}
N34 = {}
V_TIW_ts = {}

VT_TIW_ts = {}
SHFLX_ts = {}

for key in keys:
    SST_TIW_ts[key] = (SST_TIW[key].sel(x_rho=slice(TIWxreg[0],TIWxreg[1])).sel(y_rho=slice(TIWyreg[0],TIWyreg[1]))**2.).mean('x_rho').mean('y_rho')
    V_TIW_ts[key] = (V_TIW[key].sel(x_v=slice(TIWxreg[0],TIWxreg[1])).sel(y_v=slice(TIWyreg[0],TIWyreg[1]))**2.).mean('x_v').mean('y_v')
    N34[key] = SST[key].sel(x_rho=slice(-170.+360.,-120.+360.)).sel(y_rho=slice(-5.,5.)).mean('x_rho').mean('y_rho')
    
    Tt = SST_TIW[key].sel(x_rho=slice(TIWxreg[0],TIWxreg[1])).sel(y_rho=yflx,method='nearest')
    Vt = V_TIW[key].sel(x_v=slice(TIWxreg[0],TIWxreg[1])).sel(y_v=yflx,method='nearest')
    VT_TIW_ts[key] = (Tt*Vt.rename({'x_v':'x_rho'})).mean('x_rho')
    
    print(key)
    SHFLX_ts[key] = data[key]['day'].shflx.sel(x_rho=slice(-170.+360.,-120.+360.)).sel(y_rho=slice(-5.,5.)).sel(time=slice(times[0],times[1])).mean('x_rho').mean('y_rho').load()

In [ ]:
fig,axes = plt.subplots(nrows=5,ncols=1,figsize=(14,18),facecolor='w')
cols = ['C0','C1','C2']

for i, key in enumerate(keys):
    N34[key].plot(ax=axes[0],label=key,color=cols[i])
    SST_TIW_ts[key].plot(ax=axes[1],label=key,color=cols[i])
    V_TIW_ts[key].plot(ax=axes[2],label=key,color=cols[i])
    VT_TIW_ts[key].plot(ax=axes[3],label=key,color=cols[i])
    SHFLX_ts[key].plot(ax=axes[4],label=key,color=cols[i])
    
    # N34[key].resample(time="10D").mean().plot(ax=axes[0],color=cols[i],linestyle='dashed')
    # SST_TIW_ts[key].resample(time="10D").mean().plot(ax=axes[1],color=cols[i],linestyle='dashed')
    # V_TIW_ts[key].resample(time="10D").mean().plot(ax=axes[2],color=cols[i],linestyle='dashed')
    # VT_TIW_ts[key].resample(time="10D").mean().plot(ax=axes[3],color=cols[i],linestyle='dashed')
    # SHFLX_ts[key].resample(time="10D").mean().plot(ax=axes[4],color=cols[i],linestyle='dashed')
    
for axs in axes:
    axs.grid()
axes[0].legend()
axes[0].set_ylabel('Mean SST [degC]')
axes[1].set_ylabel('TIW-SST Variance [degC^2]')
axes[2].set_ylabel('TIW-V variance [m2s-2]')
axes[3].set_ylabel('TIW SST*V at ' + str(yflx) + 'N [degC ms-1]')
axes[4].set_ylabel('Mean Qnet [Wm-2]')
axes[2].set_ylim([0.,0.1])
axes[1].set_ylim([0.,0.5])
plt.tight_layout()

plt.savefig('Smooth-SST/LaNina_2016-2017_TimeSeries.png',dpi=100)

In [ ]:
fig,axes = plt.subplots(nrows=2,ncols=1,figsize=(15,10))

data['CROCO-WRF Control']['ts_3day'].T_Vmix.cumsum('s_rho').sel(time='2016-11-01',method='nearest').isel(s_rho=65).plot(ax=axes[0],vmin=-1.e-5,vmax=1.e-5)
data['CROCO-WRF Smooth-SST']['ts_3day'].T_Vmix.cumsum('s_rho').sel(time='2016-11-01',method='nearest').isel(s_rho=65).plot(ax=axes[1],vmin=-1.e-5,vmax=1.e-5)
#plt.gca().set_xlim([180.,280.])
#plt.gca().set_ylim([-6.,9.])

In [ ]:
fig,axes = plt.subplots(nrows=6,ncols=2,figsize=(30,30))
axs = axes.reshape(-1)

for i in range(12):
    print(i)
    SST1 = data['CROCO-WRF Control']['mon'].shflx.isel(time=i)
    SST2 = data['CROCO-WRF Smooth-SST']['mon'].shflx.isel(time=i)
    (SST2-SST1).plot(ax=axs[i],vmin=-50.,vmax=50.,cmap='RdBu_r')
    axs[i].set_title('CROCO-WRF Smooth-SST - CROCO-WRF Control Qnet 2016 Month ' + str(i+1))
plt.tight_layout()
#plt.gca().set_xlim([180.,280.])
#plt.gca().set_ylim([-6.,9.])

In [ ]:
time = '2016-11-15'
fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(25,8))
axs = axes.reshape(-1)

TsqCONT = data['CROCO-WRF Control']['mon'].temp_sq.isel(s_rho=-1)-data['CROCO-WRF Control']['mon'].temp.isel(s_rho=-1)**2.
TsqSMTH = data['CROCO-WRF Smooth-SST']['mon'].temp_sq.isel(s_rho=-1)-data['CROCO-WRF Smooth-SST']['mon'].temp.isel(s_rho=-1)**2.

TsqCONT.isel(time=slice(8,11)).mean('time').plot(ax=axs[0],vmin=0.,vmax=2.,cmap=cm.cm.amp)
axs[0].set_title('CROCO-WRF Control submontly SST Variability Oct-Dec 2016 ($^\circ$C$^2$)')
TsqSMTH.isel(time=slice(8,11)).mean('time').plot(ax=axs[1],vmin=0.,vmax=2.,cmap=cm.cm.amp)
axs[1].set_title('CROCO-WRF Smooth-SST submontly SST Variability Oct-Dec 2016 ($^\circ$C$^2$)')
plt.tight_layout()
#plt.gca().set_xlim([180.,280.])
#plt.gca().set_ylim([-6.,9.])